In [ ]:
import pandas as pd

# import scipy
# from scipy.stats import t
# import seaborn as sns
# from datetime import datetime, timedelta
# import plotly.express as px
# from pathlib import Path
# import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.multitest import multipletests
# from scipy.stats import f_oneway as anova
# from statsmodels.formula.api import ols 
  
# import math
# from select import select

#kernal must be restarted if these module change 

from SCP_processor import SCP_processor
processor = SCP_processor(ignore_peptides=True)
from SCP_plotter import SCP_plotter
plotter = SCP_plotter(write_output = True, processor=processor)

In [ ]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
filelist = [
             {"input1":"input/rep1-5/report.pg_matrix.tsv",
             "input2":"input/rep1-5/report.pr_matrix.tsv",
             "input3":"input/rep1-5/report.pg_matrix.tsv",
             "input4":"input/rep1-5/report-first-pass.pr_matrix.tsv",
             "input5":"Input/Payne2/Payne_organoids2.0_InputFiles.txt",
             "process_app": "DIANN_LF"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= inpit_files


SETTINGS_FILE = "HYE_200pg.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [ ]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = processor.read_files(grouped_input_files=filelist,organisms=["HUMAN","YEAST","ECOLI"])




In [ ]:
data_obj = processor.outer_join_data_objects(x)

In [ ]:

settings_table = pd.read_table(SETTINGS_FILE,sep="\t")
saved_settings = settings_table.set_index("Conditions").to_dict(orient="index")
#any run with any of the filter_out items will not be used.

for eachGroup in saved_settings:
    i = 0
    saved_settings[eachGroup]["records"] = []
    filterOutType = type(saved_settings[eachGroup]["filter_out"])
    if filterOutType == str or filterOutType == int or filterOutType == float and not pd.isna(saved_settings[eachGroup]["filter_out"]):
        filterOut = str.split(saved_settings[eachGroup]["filter_out"],sep = ",")
    else:
        filterOut = ["M@di"]
    if len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) > 1: #multiple files, only some have the runs for this group
        user_list = []
        #add all runs from all analyses to be probed
        for each_fileID in str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[1:]:
            for eachIdentifier in data_obj["run_metadata"]["Run Identifier"]:    
                currentRun = data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == eachIdentifier]["Run Identifier"] 
                if currentRun.size == 1:
                    if each_fileID == str.split(eachIdentifier,sep="-")[0] and list(currentRun)[0] not in user_list:
                        user_list.append(eachIdentifier)
                        # print(list(currentRun)[0])
                        
                else:
                    pass
        #filter for runs that relate to this gorup within those analyses
        for run_id in user_list:
            # print(run_id,end="")
            run_name = list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == run_id]["Run Names"])[0]
            # print(run_name)
            if str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[0] in run_name and (not any(item in run_name for item in filterOut)):
                saved_settings[eachGroup]["records"].append(list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == run_id]["Run Identifier"])[0]) 
                print(run_id,end="")
            else:
                print(run_name,end="")   
                pass
    # elif len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) == 1: #across all files or maybe there is only one
    #         for run_name in data_obj["run_metadata"]["Run Names"]:
    #             if str(saved_settings[eachGroup]["filter_in"]) in run_name and (not any(item in run_name for item in filterOut)):
    #                 saved_settings[eachGroup]["records"].append(data_obj["run_metadata"]["Run Names"][i])  
    #             i = i + 1   

#add the order of each column
ignore_columns = ["filter_in","filter_out"]
category_columns = [x for x in settings_table.columns.to_list() if x not in ignore_columns]

for eachCol in category_columns:
    saved_settings["Order@"+eachCol] = settings_table[eachCol].drop_duplicates().to_list()
    

In [ ]:
display(data_obj["run_metadata"])
print(saved_settings)
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "protein Identification (MBR)",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ID mode": "grouped_stacked",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Organism", #ID_Mode does MS2 vs MBR
            "Group By Color": "Amount",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.ID_plots(data_obj, plot_options, saved_settings)
figure.show()

In [ ]:
# CV violin plot
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [-10, 400],
        "CV mode": "grouped_stacked",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Organism", #ID_Mode does MS2 vs MBR
        "Group By Color": "Amount",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.CV_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein HYE plot
plot_options={
            "Group_By_Y": "Setup",
            "Group_By_Color": "Organism",
            "colors": ["red","green","blue","black"],
            "Group_By_Amount": "Amount",
            "title": "",
            "X Title": "Log2(B)",
            "Y Title": "Log2(A/B)",
            "width": 1000,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure,box, _, _ =plotter.HYE_plots(data_obj, plot_options, saved_settings)
figure.show()
box.show()

In [ ]:
# ID Venns plot
plot_options={
            "compare groups": ["human a","yeast a","bacteria a"],
            "title": "Venn Diagram (peptide)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.venns_plots(data_obj, plot_options, saved_settings)
figure.show()



In [ ]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["human a","human b"],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.volcano_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["human a","human b"],
            "title": "PCA Analysis",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.PCA_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance Heat Map
plot_options={
            "compare groups":  ["chicken","cow"],
            "log2_transform": False,
            "significant_only": True,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =plotter.heatmap_plots(data_obj, plot_options, saved_settings)
figure.show()